Andrey_Bondarchuk_stepik_id_346032105

https://colab.research.google.com/drive/1g3m3QQcp0KBajNqALiY0Bqgm4IXO35PF?usp=sharing

<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Задание 3

## Классификация текстов

В этом задании вам предстоит попробовать несколько методов, используемых в задаче классификации, а также понять насколько хорошо модель понимает смысл слов и какие слова в примере влияют на результат.

In [ ]:
import pandas as pd
import numpy as np
import torch

from torchtext.legacy import datasets
from torchtext.legacy import data

from torchtext.legacy.data import Field, LabelField
from torchtext.legacy.data import BucketIterator

from torchtext.vocab import Vectors, GloVe

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from tqdm.autonotebook import tqdm


In [ ]:
!pip install torchinfo
#https://github.com/TylerYep/torchinfo

In [ ]:
from torchinfo import summary

In [ ]:
from sklearn.metrics import f1_score 
import math

В этом задании мы будем использовать библиотеку torchtext. Она довольна проста в использовании и поможет нам сконцентрироваться на задаче, а не на написании Dataloader-а.

In [ ]:
TEXT = Field(sequential=True, lower=True, include_lengths=True)  # Поле текста
LABEL = LabelField(dtype=torch.float)  # Поле метки

In [ ]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Датасет на котором мы будем проводить эксперементы это комментарии к фильмам из сайта IMDB.

In [ ]:
train, test = datasets.IMDB.splits(TEXT, LABEL)  # загрузим датасет
train, valid = train.split(random_state=random.seed(SEED))  # разобьем на части

downloading aclImdb_v1.tar.gz


100%|██████████| 84.1M/84.1M [00:08<00:00, 10.4MB/s]


In [ ]:
TEXT.build_vocab(train)
LABEL.build_vocab(train)

In [ ]:
# display lenght of test and traing data
print(f"Number of training examples: {len(train.examples)}")
print(f"Number of testing examples: {len(valid.examples)}")

Number of training examples: 17500
Number of testing examples: 7500


In [ ]:
INPUT_DIM = len(TEXT.vocab)
OUTPUT_DIM = len(LABEL.vocab)

In [ ]:
# размер словаря
print(INPUT_DIM)

202718


In [ ]:
# display single example at index 0
print(vars(train.examples[0]))

{'text': ['what', 'an', 'incredible', 'fall', 'for', 'sean', 'ellis.<br', '/><br', '/>you', 'gather', 'a', 'bunch', 'of', 'your', 'friends', 'at', 'home,', 'all', 'hyped', 'about', 'the', 'follow', 'up', 'work', 'of', 'sean', 'ellis.', 'you', 'have', 'an', 'vague', 'idea', 'of', 'the', 'plot,', 'no', 'spoilers', 'that', 'could', 'kill', 'the', 'fun,', 'very', 'high', 'expectations.<br', '/><br', '/>it', 'is', 'late', 'at', 'night,', 'perfect', 'atmosphere', 'for', 'a', 'movie', 'of', 'this', 'type.<br', '/><br', '/>15minutes', 'passes', 'and', 'you', 'start', 'telling', 'yourself', 'it', 'is', 'bound', 'to', 'pick', 'up,', 'at', '25mins', 'you', 'start', 'wondering', 'if', 'you', 'should', 'just', 'go', 'to', 'sleep', 'and', 'save', 'this', 'for', 'another', 'time', 'when', 'you', 'can', 'fully', 'appreciate', 'the', 'expected', 'not', 'existent', 'subtle', 'touches.', 'over', 'the', 'half', 'hour', 'mark', 'you', 'realize', 'half', 'of', 'your', 'your', 'hyped', 'up', 'audience', 'is'

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, valid_iter, test_iter = BucketIterator.splits(
    (train, valid, test), 
    batch_size = 64,
    sort_within_batch = True,
    device = device)

In [ ]:
device

'cuda'

### Посмотрим на данные

In [ ]:
sample = next(iter(train_iter))
sample


[torchtext.legacy.data.batch.Batch of size 64]
	[.text]:('[torch.cuda.LongTensor of size 115x64 (GPU 0)]', '[torch.cuda.LongTensor of size 64 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 64 (GPU 0)]

In [ ]:
sample.text[0].shape

torch.Size([115, 64])

In [ ]:
sample.text[0]

tensor([[ 14323,     10,     48,  ...,     46,    184,     10],
        [  5011,     41,      9,  ...,     25,   3009,      7],
        [    76,      6,    119,  ...,     90,    176,     30],
        ...,
        [    47,     29,      4,  ...,   6980, 108901,    610],
        [     5,     35,  45482,  ...,      1,      1,      1],
        [   394,   4056,  11047,  ...,      1,      1,      1]],
       device='cuda:0')

In [ ]:
sample.text[1]

tensor([115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115,
        115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115,
        115, 115, 115, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114,
        114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114,
        114, 114, 114, 114, 113, 113, 113, 113], device='cuda:0')

In [ ]:
sample.label

tensor([1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
        1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0.,
        0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0.,
        0., 1., 0., 1., 1., 1., 1., 0., 0., 0.], device='cuda:0')

In [ ]:
sample.text[1].shape

torch.Size([64])

## RNN

Для начала попробуем использовать рекурентные нейронные сети. На семинаре вы познакомились с GRU, вы можете также попробовать LSTM. Можно использовать для классификации как hidden_state, так и output последнего токена.

### Torch LSTM

Возьмем реализацию LSTM из torch

In [ ]:
class RNNBaseline(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx, use_last_token=False):
        
        super().__init__()  

        self.use_last_token = use_last_token     
   
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        # ---------------------------------------------- torch lstm ------------------------------------------------------
        self.rnn = nn.LSTM(input_size = embedding_dim, hidden_size = hidden_dim, num_layers = n_layers, batch_first=False, 
                           bidirectional=bidirectional, dropout = dropout)  # YOUR CODE GOES HERE
        # ----------------------------------------------------------------------------------------------------------------
        if bidirectional:
            self.fc = nn.Linear(2*hidden_dim, output_dim)  # YOUR CODE GOES HERE
        else:
            self.fc = nn.Linear(hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
            
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
         
        #embedded = [sent len, batch size, emb dim]
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        # cell arg for LSTM, remove for GRU
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)  

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors

        # ---- Можно использовать для классификации как hidden_state, так и output последнего токена ---
        if self.use_last_token:
            output = self.dropout(output[-1])
            output = self.fc(output)
            output = output.squeeze(1)
            return output
        # -----------------------------------------------------------------------------------------------
        
        # #hidden = [num layers * num directions, batch size, hid dim]
        # #cell = [num layers * num directions, batch size, hid dim]
        
        # #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        # #and apply dropout          
                  
        #hidden = [batch size, hid dim * num directions] or [batch_size, hid dim * num directions]
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))  # YOUR CODE GOES HERE
        hidden = self.fc(hidden)
        hidden = hidden.squeeze(1)
        return hidden

Напишем функцию для тренировки модели

In [ ]:
# велосипед
def training(model, optimizer, loss_func, train_iter, valid_iter, max_epochs):

  min_loss = np.inf
  cur_patience = 0
  max_grad_norm = 2
  threshold = 0.5

  for epoch in range(1, max_epochs + 1):
      
      train_loss = 0.0
      model.train()
      pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
      pbar.set_description(f"Epoch {epoch}")
      for it, batch in pbar: 
          #YOUR CODE GOES HERE ---------------------------------------------------------------
          features, text_lengths = batch.text
          labels = batch.label
          prediction = model(features, text_lengths.cpu())
          loss = loss_func(prediction, labels)
          loss.backward()
          train_loss += loss
          if max_grad_norm is not None:
              torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
          optimizer.step()

          features.cpu()
          del features
          labels.cpu()
          del labels 
          # ----------------------------------------------------------------------------------

      train_loss /= len(train_iter)
      
      val_loss = 0.0
      
      model.eval()
      with torch.no_grad():
          correct = 0
          num_objs = 0
          pbar = tqdm(enumerate(valid_iter), total=len(valid_iter), leave=False)
          pbar.set_description(f"Epoch {epoch}")
          for it, batch in pbar:
              # YOUR CODE GOES HERE -----------------------------------------------------------
              features, text_lengths = batch.text
              labels = batch.label
              prediction = model(features, text_lengths.cpu())
              loss = loss_func(prediction, labels)
              val_loss += loss
              correct += ((torch.sigmoid(prediction)>=threshold).float() == labels).float().sum().cpu().item()
              num_objs += len(labels)
              # ------------------------------------------------------------------------------
    
      val_loss /= len(valid_iter)
      if val_loss < min_loss:
          min_loss = val_loss
          best_model = model.state_dict()
      else:
          cur_patience += 1
          if cur_patience == patience:
              cur_patience = 0
              break
      
      print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}, Accuracy: {:.4f}'.format(epoch, train_loss, val_loss, correct/num_objs))

  model.load_state_dict(best_model)

Поиграйтесь с гиперпараметрами

In [ ]:
vocab_size = len(TEXT.vocab)
emb_dim = 100
hidden_dim = 256
output_dim = 1
n_layers = 2
bidirectional = True
dropout = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
patience=3

In [ ]:
model = RNNBaseline(
    vocab_size=vocab_size,
    embedding_dim=emb_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    n_layers=n_layers,
    bidirectional=bidirectional,
    dropout=dropout,
    pad_idx=PAD_IDX,
    use_last_token = False
)

In [ ]:
model

RNNBaseline(
  (embedding): Embedding(202718, 100, padding_idx=1)
  (rnn): LSTM(100, 256, num_layers=2, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (relu): ReLU()
)

Обучите сетку! Используйте любые вам удобные инструменты, Catalyst, PyTorch Lightning или свои велосипеды.

In [ ]:
model = model.to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_func = nn.BCEWithLogitsLoss()

max_epochs = 20

In [ ]:
%%time
training(model, optimizer, loss_func, train_iter, valid_iter, max_epochs)
print("Done")

  0%|          | 0/274 [00:00<?, ?it/s]

  0%|          | 0/118 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.6688, Validation Loss: 0.6275, Accuracy: 0.6497


  0%|          | 0/274 [00:00<?, ?it/s]

  0%|          | 0/118 [00:00<?, ?it/s]

...

  0%|          | 0/118 [00:00<?, ?it/s]

Epoch: 8, Training Loss: 0.1830, Validation Loss: 0.4974, Accuracy: 0.8031


  0%|          | 0/274 [00:00<?, ?it/s]

  0%|          | 0/118 [00:00<?, ?it/s]

Done
CPU times: user 12min 3s, sys: 1min 37s, total: 13min 41s
Wall time: 13min 42s


Сохраним веса нашей модели

In [ ]:
torch.save(model.state_dict(), "model_lstm.pt")

загрузить веса в дальнейшем можно коммандой 
``` python
model.load_state_dict(torch.load("model_lstm.pt", map_location=device))
```

#### Расчет метрики F1 на тестовом датасете

Для порогового значения вероятностей принадлежности к одному из двух классов примем значение = 0.5

In [ ]:
threshold = 0.5

In [ ]:
label_list = []
pred_list = []

model.eval()
with torch.no_grad():
    correct = 0
    num_objs = 0
    pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
    for it, batch in pbar:
        features, text_lengths = batch.text
        labels = batch.label
        label_list.append(labels.cpu())
        prediction = model(features, text_lengths.cpu())
        preds = (torch.sigmoid(prediction)>=threshold).float()
        pred_list.append(preds.cpu())
        correct += (preds == labels).float().sum().cpu().item()
        num_objs += len(labels)

accuracy = correct/num_objs

target = torch.cat(label_list, dim=-1)
predict = torch.cat(pred_list, dim=-1)

  0%|          | 0/391 [00:00<?, ?it/s]

In [ ]:
f1 = f1_score(target, predict)

print(' f1_score : {:.4f}, Accuracy: {:.4f}'.format(f1, accuracy))

 f1_score : 0.8201, Accuracy: 0.8047


Собирать данные будем в список словарей и затем результат загрузим в  pandas dataframe

In [ ]:
pd_list =[]
pd_list.append({"model": "LSTM (torch)", "f1_score": round(f1,5), "accuracy": round(accuracy,5)})

**Ответ**: **Для LSTM модели получены следующие результаты: f1_score : 0.8201, accuracy: 0.8047**

## CNN

![](https://www.researchgate.net/publication/333752473/figure/fig1/AS:769346934673412@1560438011375/Standard-CNN-on-text-classification.png)

Для классификации текстов также часто используют сверточные нейронные сети. Идея в том, что как правило сентимент содержат словосочетания из двух-трех слов, например "очень хороший фильм" или "невероятная скука". Проходясь сверткой по этим словам мы получим какой-то большой скор и выхватим его с помощью MaxPool. Далее идет обычная полносвязная сетка. Важный момент: свертки применяются не последовательно, а параллельно. Давайте попробуем!

In [ ]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
TEXT = Field(sequential=True, lower=True, batch_first=True)  # batch_first тк мы используем conv  
LABEL = LabelField(batch_first=True, dtype=torch.float)

train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

TEXT.build_vocab(trn)
LABEL.build_vocab(trn)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
train_iter, valid_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

Вы можете использовать Conv2d с `in_channels=1, kernel_size=(kernel_sizes[0], emb_dim))` или Conv1d c `in_channels=emb_dim, kernel_size=kernel_size[0]`. Но хорошенько подумайте над shape в обоих случаях.

In [ ]:
class CNN(nn.Module):
    def __init__(
        self,
        vocab_size,
        emb_dim,
        out_channels,
        kernel_sizes,
        dropout=0.5,
    ):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.conv_0 = nn.Conv1d(in_channels = emb_dim, out_channels = out_channels, kernel_size = kernel_sizes[0])  # YOUR CODE GOES HERE
        self.conv_1 = nn.Conv1d(in_channels = emb_dim, out_channels = out_channels, kernel_size = kernel_sizes[1])  # YOUR CODE GOES HERE
        self.conv_2 = nn.Conv1d(in_channels = emb_dim, out_channels = out_channels, kernel_size = kernel_sizes[2])   # YOUR CODE GOES HERE
        self.fc = nn.Linear(len(kernel_sizes) * out_channels, 1)
        self.dropout = nn.Dropout(dropout)       
        
    def forward(self, text):
        
        #text = [batch size, sent len]
        embedded = self.embedding(text)
        
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.permute(0, 2, 1)  # may be reshape here
        
        conved_0 = F.relu(self.conv_0(embedded))  # may be reshape here 
        conved_1 = F.relu(self.conv_1(embedded))  # may be reshape here
        conved_2 = F.relu(self.conv_2(embedded))  # may be reshape here
        
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))
            
        return self.fc(cat).squeeze(1)

In [ ]:
kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
out_channels=64
dropout = 0.5
dim = 300

model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=out_channels,
            kernel_sizes=kernel_sizes, dropout=dropout)

In [ ]:
input = tuple(next(iter(train_iter)).text.shape)
input

(128, 986)

In [ ]:
input = (128,1000)

In [ ]:
# посмотрим на модель
summary(model, input, dtypes=[torch.long])

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      --                        --
├─Embedding: 1-1                         [128, 1000, 300]          60,815,400
├─Conv1d: 1-2                            [128, 64, 998]            57,664
├─Conv1d: 1-3                            [128, 64, 997]            76,864
├─Conv1d: 1-4                            [128, 64, 996]            96,064
├─Dropout: 1-5                           [128, 192]                --
├─Linear: 1-6                            [128, 1]                  193
Total params: 61,046,185
Trainable params: 61,046,185
Non-trainable params: 0
Total mult-adds (G): 37.21
Input size (MB): 1.02
Forward/backward pass size (MB): 503.22
Params size (MB): 244.18
Estimated Total Size (MB): 748.43

In [ ]:
model.to(device)

CNN(
  (embedding): Embedding(202718, 300)
  (conv_0): Conv1d(300, 64, kernel_size=(3,), stride=(1,))
  (conv_1): Conv1d(300, 64, kernel_size=(4,), stride=(1,))
  (conv_2): Conv1d(300, 64, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=192, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),  lr=0.0001)
loss_func = nn.BCEWithLogitsLoss()

In [ ]:
max_epochs = 30

Обучите!

In [ ]:
# велосипед для Convolutional Neural Networks for Sentence Classification
def training_cnn(model, optimizer, loss_func, train_iter, valid_iter, max_epochs):

  min_loss = np.inf
  cur_patience = 0
  max_grad_norm = 2
  patience=3
  threshold = 0.5

  for epoch in range(1, max_epochs + 1):
      
      train_loss = 0.0
      model.train()
      pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
      pbar.set_description(f"Epoch {epoch}")
      for it, batch in pbar: 
          #YOUR CODE GOES HERE ---------------------------------------------------------------
          features = batch.text
          labels = batch.label
          prediction = model(features)
          loss = loss_func(prediction, labels)
          loss.backward()
          train_loss += loss
          if max_grad_norm is not None:
              torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
          optimizer.step()

          features.cpu()
          del features
          labels.cpu()
          del labels 
          # ----------------------------------------------------------------------------------

      train_loss /= len(train_iter)
      
      val_loss = 0.0
      
      model.eval()
      with torch.no_grad():
          correct = 0
          num_objs = 0
          pbar = tqdm(enumerate(valid_iter), total=len(valid_iter), leave=False)
          pbar.set_description(f"Epoch {epoch}")
          for it, batch in pbar:
              # YOUR CODE GOES HERE -----------------------------------------------------------
              features = batch.text
              labels = batch.label
              prediction = model(features)
              loss = loss_func(prediction, labels)
              val_loss += loss
              correct += ((torch.sigmoid(prediction)>=threshold).float() == labels).float().sum().cpu().item()
              num_objs += len(labels)
              # ------------------------------------------------------------------------------
    
      val_loss /= len(valid_iter)
      if val_loss < min_loss:
          min_loss = val_loss
          best_model = model.state_dict()
      else:
          cur_patience += 1
          if cur_patience == patience:
              cur_patience = 0
              break
      
      print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}, Accuracy: {:.4f}'.format(epoch, train_loss, val_loss, correct/num_objs))

  model.load_state_dict(best_model)

In [ ]:
%%time
training_cnn(model, optimizer, loss_func, train_iter, valid_iter, max_epochs)
print('Done')

  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.7463, Validation Loss: 0.6082, Accuracy: 0.7149


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 19, Training Loss: 0.1744, Validation Loss: 0.3486, Accuracy: 0.8504


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Done
CPU times: user 16min 59s, sys: 6.23 s, total: 17min 5s
Wall time: 17min 8s


In [ ]:
torch.save(model.state_dict(), "model_cnn.pt")

##### Расчет метрики F1 на тестовом датасете

In [ ]:
label_list = []
pred_list = []

model.eval()
with torch.no_grad():
    correct = 0
    num_objs = 0
    pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
    for it, batch in pbar:
        features = batch.text
        labels = batch.label
        label_list.append(labels.cpu())
        prediction = model(features)
        preds = (torch.sigmoid(prediction)>=threshold).float()
        pred_list.append(preds.cpu())
        correct += (preds == labels).float().sum().cpu().item()
        num_objs += len(labels)

accuracy = correct/num_objs

target = torch.cat(label_list, dim=-1)
predict = torch.cat(pred_list, dim=-1)

  0%|          | 0/98 [00:00<?, ?it/s]

In [ ]:
f1 = f1_score(target, predict)
print(' f1_score : {:.4f}, accuracy: {:.4f}'.format(f1, accuracy))

 f1_score : 0.8567, accuracy: 0.8452


In [ ]:
pd_list.append({"model": "CNN", "f1_score": round(f1,5), "accuracy": round(accuracy,5)})

In [ ]:
pd_list

[{'accuracy': 0.80472, 'f1_score': 0.8201, 'model': 'LSTM (torch)'},
 {'accuracy': 0.84516, 'f1_score': 0.85673, 'model': 'CNN'}]

**Ответ**: **Для CNN модели получены следующие результаты: f1_score : 0.8567, accuracy: 0.8452**

## Интерпретируемость

Посмотрим, куда смотрит наша модель. Достаточно запустить код ниже.

In [ ]:
!pip install -q captum

     |████████████████████████████████| 1.4 MB 4.3 MB/s 


In [ ]:
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)

In [ ]:
def forward_with_softmax(inp):
    logits = model(inp)
    return torch.softmax(logits, 0)[0][1]

def forward_with_sigmoid(input):
    return torch.sigmoid(model(input))


# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 7, label = 0):
    model.eval()
    text = [tok for tok in TEXT.tokenize(sentence)]
    if len(text) < min_len:
        text += ['pad'] * (min_len - len(text))
    indexed = [TEXT.vocab.stoi[t] for t in text]

    model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    # input_indices dim: [sequence_length]
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid(input_indices).item()
    pred_ind = round(pred)

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
                                           n_steps=5000, return_convergence_delta=True)

    print('pred: ', LABEL.vocab.itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            LABEL.vocab.itos[pred_ind],
                            LABEL.vocab.itos[label],
                            LABEL.vocab.itos[1],
                            attributions.sum(),       
                            text,
                            delta))

In [ ]:
interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.88 ) , delta:  tensor([6.2750e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.00 ) , delta:  tensor([8.8248e-06], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.96 ) , delta:  tensor([7.8626e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.01 ) , delta:  tensor([6.6836e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.17 ) , delta:  tensor([4.3947e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.08 ) , delta:  tensor([1.5667e-05], device='cuda:0', dtype=torch.float64)


Попробуйте добавить свои примеры!

In [ ]:
interpret_sentence(model, 'It is a dangerous movie!', label=0)

pred:  pos ( 0.68 ) , delta:  tensor([3.3287e-05], device='cuda:0', dtype=torch.float64)


In [ ]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.88),pos,1.58,It was a fantastic performance ! pad
pos,neg (0.00),pos,1.19,Best film ever pad pad pad pad
pos,pos (0.96),pos,1.46,Such a great show! pad pad pad
neg,neg (0.01),pos,0.31,It was a horrible movie pad pad
neg,neg (0.17),pos,1.13,I've never watched something as bad pad
neg,neg (0.08),pos,1.01,It is a disgusting movie! pad pad
neg,pos (0.68),pos,1.93,It is a dangerous movie! pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.88),pos,1.58,It was a fantastic performance ! pad
pos,neg (0.00),pos,1.19,Best film ever pad pad pad pad
pos,pos (0.96),pos,1.46,Such a great show! pad pad pad
neg,neg (0.01),pos,0.31,It was a horrible movie pad pad
neg,neg (0.17),pos,1.13,I've never watched something as bad pad
neg,neg (0.08),pos,1.01,It is a disgusting movie! pad pad
neg,pos (0.68),pos,1.93,It is a dangerous movie! pad pad


## Эмбеддинги слов

Вы ведь не забыли, как мы можем применить знания о word2vec и GloVe. Давайте попробуем!

In [ ]:
train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

#### CNN + embedding

In [ ]:
TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=300)) # YOUR CODE GOES HERE
# подсказка: один из импортов пока не использовался, быть может он нужен в строке выше :)
LABEL.build_vocab(trn)

word_embeddings = TEXT.vocab.vectors

kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
dropout = 0.5
dim = 300

.vector_cache/glove.6B.zip: 862MB [02:42, 5.29MB/s]                           
100%|█████████▉| 399999/400000 [00:51<00:00, 7721.48it/s]


In [ ]:
model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=64,
            kernel_sizes=kernel_sizes, dropout=dropout)

pretrained_embeddings = TEXT.vocab.vectors

prev_shape = model.embedding.weight.shape

# инициализируйте эмбэдинг
model.embedding.weight.data.copy_(pretrained_embeddings)

assert prev_shape == model.embedding.weight.shape

model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_func = nn.BCEWithLogitsLoss()

Вы знаете, что делать.

In [ ]:
%%time
training_cnn(model, optimizer, loss_func, train_iter, valid_iter, max_epochs)
print('Done')

  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.6496, Validation Loss: 0.5637, Accuracy: 0.7563


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 8, Training Loss: 0.1407, Validation Loss: 0.3472, Accuracy: 0.8507


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Done
CPU times: user 7min 46s, sys: 3.06 s, total: 7min 49s
Wall time: 8min 1s


In [ ]:
torch.save(model.state_dict(), "model_cnn_emb.pt")

##### Расчет метрики F1 на тестовом датасете

In [ ]:
label_list = []
pred_list = []

model.eval()
with torch.no_grad():
    correct = 0
    num_objs = 0
    pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
    for it, batch in pbar:
        features = batch.text
        labels = batch.label
        label_list.append(labels.cpu())
        prediction = model(features)
        preds = (torch.sigmoid(prediction)>=threshold).float()
        pred_list.append(preds.cpu())
        correct += (preds == labels).float().sum().cpu().item()
        num_objs += len(labels)

accuracy = correct/num_objs

target = torch.cat(label_list, dim=-1)
predict = torch.cat(pred_list, dim=-1)

  0%|          | 0/98 [00:00<?, ?it/s]

In [ ]:
f1 = f1_score(target, predict)
print(' f1_score : {:.4f}, accuracy: {:.4f}'.format(f1, accuracy))

 f1_score : 0.8111, accuracy: 0.8312


Посчитайте f1-score вашего классификатора.

**Ответ**: **Для CNN c загрузкой предобученных ембеддингов получены следующие результаты: f1_score : 0.8111, accuracy: 0.8312**

In [ ]:
pd_list.append({"model": "CNN + emb", "f1_score": round(f1,5), "accuracy": round(accuracy,5)})

In [ ]:
pd_list

[{'accuracy': 0.80472, 'f1_score': 0.8201, 'model': 'LSTM (torch)'},
 {'accuracy': 0.84516, 'f1_score': 0.85673, 'model': 'CNN'},
 {'accuracy': 0.83124, 'f1_score': 0.81112, 'model': 'CNN + emb'}]

Проверим насколько все хорошо!

In [ ]:
PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)
vis_data_records_ig = []

interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.90 ) , delta:  tensor([7.8264e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.04 ) , delta:  tensor([4.0485e-06], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.23 ) , delta:  tensor([1.4978e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.00 ) , delta:  tensor([5.4480e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.28 ) , delta:  tensor([0.0001], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.00 ) , delta:  tensor([1.0710e-06], device='cuda:0', dtype=torch.float64)


In [ ]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.90),pos,1.86,It was a fantastic performance ! pad
pos,neg (0.04),pos,0.64,Best film ever pad pad pad pad
pos,neg (0.23),pos,1.41,Such a great show! pad pad pad
neg,neg (0.00),pos,-0.58,It was a horrible movie pad pad
neg,neg (0.28),pos,1.13,I've never watched something as bad pad
neg,neg (0.00),pos,-0.56,It is a disgusting movie! pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.90),pos,1.86,It was a fantastic performance ! pad
pos,neg (0.04),pos,0.64,Best film ever pad pad pad pad
pos,neg (0.23),pos,1.41,Such a great show! pad pad pad
neg,neg (0.00),pos,-0.58,It was a horrible movie pad pad
neg,neg (0.28),pos,1.13,I've never watched something as bad pad
neg,neg (0.00),pos,-0.56,It is a disgusting movie! pad pad


#### CNN + freeze embedding

Посмотрим, что получиться если заморозить ембеддинги

In [ ]:
model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=64,
            kernel_sizes=kernel_sizes, dropout=dropout)

pretrained_embeddings = TEXT.vocab.vectors

prev_shape = model.embedding.weight.shape

# инициализируйте эмбэдинг
model.embedding.weight.data.copy_(pretrained_embeddings)

assert prev_shape == model.embedding.weight.shape

model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_func = nn.BCEWithLogitsLoss()

In [ ]:
# заморозка 
embeddings = model.embedding
for p in embeddings.parameters():
    p.requires_grad = False

In [ ]:
%%time
training_cnn(model, optimizer, loss_func, train_iter, valid_iter, max_epochs)
print('Done')

  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.6587, Validation Loss: 0.5762, Accuracy: 0.7835


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 12, Training Loss: 0.2804, Validation Loss: 0.3504, Accuracy: 0.8429


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Done
CPU times: user 7min 27s, sys: 3.64 s, total: 7min 31s
Wall time: 7min 50s


In [ ]:
torch.save(model.state_dict(), "model_cnn_emb_freeze.pt")

##### Расчет метрики F1 на тестовом датасете

In [ ]:
label_list = []
pred_list = []

model.eval()
with torch.no_grad():
    correct = 0
    num_objs = 0
    pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
    for it, batch in pbar:
        features = batch.text
        labels = batch.label
        label_list.append(labels.cpu())
        prediction = model(features)
        preds = (torch.sigmoid(prediction)>=threshold).float()
        pred_list.append(preds.cpu())
        correct += (preds == labels).float().sum().cpu().item()
        num_objs += len(labels)

accuracy = correct/num_objs

target = torch.cat(label_list, dim=-1)
predict = torch.cat(pred_list, dim=-1)

  0%|          | 0/98 [00:00<?, ?it/s]

In [ ]:
f1 = f1_score(target, predict)
print(' f1_score : {:.4f}, accuracy: {:.4f}'.format(f1, accuracy))

 f1_score : 0.8433, accuracy: 0.8520


In [ ]:
pd_list.append({"model": "CNN + freeze emb", "f1_score": round(f1,5), "accuracy": round(accuracy,5)})

**Ответ**: **Для CNN c загрузкой предобученных ембеддингов и последующей заморозкой получены следующие результаты: f1_score : 0.8433, accuracy: 0.8520**

Соберем все полученные результаты в итоговую табличку.

In [ ]:
import pandas as pd
pd_result = pd.DataFrame(pd_list).sort_values(by=['f1_score'], ascending=False)
pd_result.to_csv('result.csv', index=False)  

# Итоги исследования

Были построены четыре различные модели для решения задачи классификации тональности текстов.

- **LSTM** - модель, в которой использовалась torch реализация реккурентной ячейки lstm.
- **CNN** - модель, с параллельной обработкой в три протока сверток с ядрами [3, 4, 5] и неинициализированными ембеддингами.
- **CNN + emb** - модель, с параллельной обработкой в три протока сверток с ядрами [3, 4, 5] и инициализированными ембеддингами. Инициализация была взята из библиотеки GloVe(name='6B', dim=300) В дальнейшем данная модель сама дообучала ембеддинги.
- **CNN + freeze emb** - - модель, с параллельной обработкой в три протока сверток с ядрами [3, 4, 5] и инициализированными ембеддингами от GloVe. Ембеддинги модели были заморожены.

Полученные результаты сведены в итоговую таблицу:

In [ ]:
pd_result

,model,f1_score,accuracy
1,CNN,0.85673,0.84516
3,CNN + freeze emb,0.84329,0.85196
0,LSTM (torch),0.82010,0.80472
2,CNN + emb,0.81112,0.83124


**Наилучшие результаты по метрике f1 были получены на сверточной модели CNN. В данной модели ембеддинги обучались изначально под свой датасет.**